## Analyzing Atrato's Portuguese Migrants 🇵🇹

In [76]:
import pandas as pd
import geopandas as gpd
import numpy as np
import pylab as pl
import matplotlib.pyplot as plt
%matplotlib inline

In [77]:
df = pd.read_excel('atrato-data.xlsx')
df = df[df.nationality=="Portuguese"]

### Demographics

In [78]:
display("Total of Portuguese migrants:{}".format(len(df)))

TypeError: 'module' object is not callable

In [ ]:
gender = df["gender"].to_frame("Gender").value_counts().to_frame("Total")
gender.plot.pie(title="Gender Distribution", figsize=(3,3), colors = ['green','red'],
                autopct=(lambda prct_value: '{:.1f}%\n{:.0f}'.format(prct_value,len(df)*prct_value/100)),
                shadow=True, startangle=90, labels=["Male","Female"], y='Total').get_legend().remove()

### Travel

In [ ]:
passport = df["found_passport"].to_frame("Passports Found").value_counts().to_frame("Total")
passport.plot.pie(title="Passports", figsize=(3,3), colors = ('steelblue','plum'),
                autopct=(lambda prct_value: '{:.1f}%\n{:.0f}'.format(prct_value,len(df)*prct_value/100)),
                shadow=True, startangle=90, labels=["Found","Not Found"], y='Total').get_legend().remove()

In [ ]:
distritos_geodata = gpd.read_file("https://raw.githubusercontent.com/nmota/caop_GeoJSON/master/ContinenteDistritos.geojson")
distritos_data = pd.read_csv("naturality_distritos.csv", converters={'DI': str})

In [ ]:
distritos_map = distritos_geodata.merge(distritos_data, on="DI")
distritos_map = distritos_map[["DI", "geometry", "counter"]]
distritos_map.plot(column = "counter",cmap='YlGnBu',figsize=(16,10),  legend=True,edgecolor="black")
display.display(pl.gcf())
display.clear_output(wait=True)
plt.close()